In [4]:
from flask import Flask, request, jsonify, render_template
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

app = Flask(__name__)

# Sample data (replace this with your actual data loading logic)
data = {
    'Title': ['Book 1', 'Book 2', 'Book 3'],
    'Author': ['Author 1', 'Author 2', 'Author 3'],
    'Genre': ['Science', 'Science', 'Science'],
    'SubGenre': ['Astronomy', 'Physics', 'Mathematics']
}
df2 = pd.DataFrame(data)

# Prepare data and compute cosine similarity matrix
def prepare_data(df2):
    df2['combined_features'] = df2['Genre'] + ' ' + df2['SubGenre'].fillna('')
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(df2['combined_features'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return df2, cosine_sim

df2, cosine_sim = prepare_data(df2)

# Function to get recommendations
def recommend_books(df2, cosine_sim, selected_genre=None, selected_subgenre=None, num_recommendations=3):
    if selected_genre and selected_subgenre:
        genre_books = df2[df2['Genre'].str.contains(selected_genre, case=False, na=False) &
                          df2['SubGenre'].str.contains(selected_subgenre, case=False, na=False)]
    elif selected_genre:
        genre_books = df2[df2['Genre'].str.contains(selected_genre, case=False, na=False)]
    elif selected_subgenre:
        genre_books = df2[df2['SubGenre'].str.contains(selected_subgenre, case=False, na=False)]
    else:
        return "Please select at least one genre or subgenre."

    if genre_books.empty:
        return "No books found for the selected genre and/or subgenre."

    reference_book_idx = random.choice(genre_books.index)
    sim_scores = list(enumerate(cosine_sim[reference_book_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_recommendations+1]
    book_indices = [i[0] for i in sim_scores]
    return df2.iloc[book_indices].to_dict('records')

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.get_json()
    selected_genre = data.get('genre')
    selected_subgenre = data.get('subgenre')
    recommendations = recommend_books(df2, cosine_sim, selected_genre, selected_subgenre, num_recommendations=3)
    return jsonify(recommendations)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\asus i5\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
py app.py

SyntaxError: invalid syntax (947945564.py, line 1)